# Fake News Classifier Final Report

Names:

# Introduction

- What is our project?
- What is the relevance?
- What is the data?
- What are our methods?

# Importing Necessary Libraries

In [ ]:
# libraries for loading, preprocessing, visualization
import pandas as np
import numpy as np

import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# libraries for NLTK
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import treebank
from nltk.stem import PorterStemmer

# libraries for spaCy
import spacy
from spacy import displacy


# libraries for Gensim
import gensim
from gensim.corpora import Dictionary

# libraries for ML models
from scikitlearn.model_selection import train_test_split


# libraries for SHAP + LIME
import shap


In [ ]:
# import dataset
df = pd.read_csv('')

# Exploratory Data Analysis

- look at missing values
- look at size
- look at data type
- look at distribution of type
- add Mahbuba's visualizations

- WHAT DID WE LEARN FROM EDA?

# Text Cleaning and Preprocessing

- drop na
- lower case
- remove stop words
- 

# Natural Language Processing

- Why do we do nlp? what are the benefits? why do we need it to build out fake news classifier?

## NLTK

- mention that it is our baseline
- pros and cons?
- what it does, how it works, what are some of its features?

## spaCy

tokenize, lemmatize, POS tagging, entity recognition

### spaCy Tokenizing

### spaCy Lemmatizing

### spaCy Part of Speech Tagging

### spaCy Entity Recognition

### spaCy Topic Modeling

## Gensim

# Building Classification Models

- decision tree
- knn
- XGBoost
- 

# Evaluating Model Performance

- what metrics are we using to evaluate models? how do they work? how are they different? why did we pick that particular evaluation metric?

- ALSO SHOW WHAT EXACTLY WAS MISCLASSIFIED, AND PLOT IT?

- ADD PLOTS COMPARING THE PERFORMANCE OF EACH MODEL

# Model Explainability

- why is model explainability important?

## SHAP

- what is it, how does it work?

- What do the result show, or what do they mean?

## LIME

- what is it, how does it work?
- run lime multiple times and on a larger sample size

- What do the result show, or what do they mean?

# Conclusion

- Short overview of whole project: why? what? how?
- where the data was found, what is its structure
- how the data was pre-processed and cleaned
- language processing techniques and what they showed us, how are they different?
- models, how they performed, how are they unique
- model explainability, how did the model make decisions?
- model evaluation, what does it mean in the greater context?
- conclusion